In [1]:
#libs initialization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import *
import scipy
from scipy import optimize
from scipy.optimize import minimize
from tqdm.notebook import tqdm 
import math

#parameters
alpha = random.uniform (0, 1)
beta = random.uniform (0, 1)

y = []
x = []
a = []
b = []
x_lma = np.arange(0, 101, 1) / 100
y_lma = alpha * x_lma + beta + random.normalvariate (0.5, 0.125)

#functions creation
def func (x):
    y = alpha * x + beta + random.normalvariate (0.5, 0.125)
    return y

#Arrays filling loops
for k in range (0, 101, 1):
    y.append( func (k / 100 ))
    x.append(k/100)

xk = np.array(x)
yk = np.array(y)

In [2]:
#Gradient descent
a = 0.3
b = 0.3
ab_0 = [a, b]
ab_i=[[0.25,0.25]]
ab_i.append(ab_0)
beta_i = []
grad_i = [[0.1, 0.1]]
i=1
while abs(ab_i[i][0] - ab_i[i-1][0]) > 0.001 and abs(ab_i[i][1] - ab_i[i-1][1]) > 0.001:
    grad = [(np.sum(2 * xk * (ab_i[i][0] * xk + ab_i[i][1] - yk)))/len(xk), (np.sum(2 * (ab_i[i][0] * xk + ab_i[i][1] - yk)))/len(xk)]
    grad_i.append(grad)
    beta = abs((np.array(ab_i[i])-np.array(ab_i[i-1]))*(np.array(grad_i[i])-np.array(grad_i[i-1])))/(abs(np.array(grad_i[i])-np.array(grad_i[i-1])))**2
    beta_i.append(beta.tolist())
    x_next = np.array(ab_i[i]) - beta_i[i-1] * np.array(grad_i[i])
    ab_i.append(x_next.tolist())
    i += 1
ex_1_lin = ab_i[i]

a = 6
b = -0.3
ab_0 = [a, b]
ab_i=[[1,-0.5]]
ab_i.append(ab_0)
beta_i = []
grad_i = [[0.1, 0.1]]
i=1
while abs(ab_i[i][0] - ab_i[i-1][0]) > 0.001 and abs(ab_i[i][1] - ab_i[i-1][1]) > 0.001:
    grad = [(np.sum((ab_i[i][0]/(1+ab_i[i][1]*xk)-yk)*2/(1+ab_i[i][1]*xk)))/len(xk), (np.sum(2*ab_i[i][0]*xk/(1+ab_i[i][1]*xk)**2*(ab_i[i][0]/(1+ab_i[i][1]*xk)-yk)))/len(xk)]
    grad_i.append(grad)
    beta = abs((np.array(ab_i[i])-np.array(ab_i[i-1]))*(np.array(grad_i[i])-np.array(grad_i[i-1])))/(abs(np.array(grad_i[i])-np.array(grad_i[i-1])))**2
    beta_i.append(beta.tolist())
    x_next = np.array(ab_i[i]) - beta_i[i-1] * np.array(grad_i[i])
    ab_i.append(x_next.tolist())
    i += 1
ex_1_rat = ab_i[i]


In [3]:
#Conjugate gradient 
def golden(func, limits, eps):
    a, b = limits
    flag = 0
    iteration = 0
    evaluation = 0
    while abs(a - b) >= eps:
        iteration += 1
        delta = (3 - math.sqrt(5)) * (b - a) / 2
        if flag == 0:
            evaluation += 2
            x1, x2 = a + delta, b - delta
            f1, f2 = func( x1 ), func( x2 )
        if flag == "x2=x1":
            evaluation += 1
            x1, x2 = a + delta, x1
            f1, f2 = func( x1 ), f1
        if flag == "x1=x2":
            evaluation += 1
            x1, x2 = x2, b - delta
            f1, f2 = f2, func( x2 )

        if f1 <= f2:
            b = x2
            flag = 'x2=x1'
        else:
            a = x1
            flag = 'x1=x2'     
    min_x = (a + b) / 2
    min_y = (f1 + f2)  / 2
    return min_x, min_y, iteration, evaluation
def conjugate_gradient(loss, gradient, start, tol=1e-3, max_iter=1000):
    vector = start
    diff = -gradient(vector)
    def alpha_min(alpha, func, diff, vector):
        return func(vector + alpha * diff)
    f_min = lambda alpha: alpha_min(alpha, loss, diff, vector)
    alpha = golden(f_min, limits=(1e-3, 1e1), eps=1e-3)[0]
    vector += alpha * diff
    diff0 = diff
    iterations = 0
    for _ in tqdm(range(max_iter)):
        iterations += 1
        diff = -gradient(vector)
        betta = diff.T * diff / diff0.T / diff0
        diff += betta * diff0
        if np.all( np.abs(diff) <=  tol):
            print("Criterion stop, iterations = ", iterations)
            break
        alpha = golden(f_min, limits=(1e-3, 1e1), eps=1e-3)[0]
        vector += alpha * diff
        diff0 = diff
    return vector


In [4]:
def lin(ab):
    a, b = ab
    s = 0
    for k in range(0, 101):
        s += (a * x[k] + b - y[k]) ** 2
    return s
def lin_jac(ab):
    a, b = ab
    l = 0
    r = 0
    s = 0
    for k in range(0, 101):
        l += (2 * x[k] * (b + a * x[k] - y[k]))
        r += (2 * (b + a * x[k] - y[k]))
    s = np.array([l, r])
    return s
def lin_hess(ab):
    a, b = ab
    hess = np.ones([2,2])
    lh = 0
    rh = 0
    ld = 0
    rd = 0
    for k in range(0, 101):
        lh += (2 * x[k]**2)
        rh += (2 * x[k])
        ld += (2 * x[k])
        rd = (2)
    hess[0,0] = lh
    hess[0,1] = rh
    hess[1,0] = ld
    hess[1,1] = rd
    return hess
def lin_lma(ab):
    a, b = ab
    global x_lma, y_lma
    return ( ((a * x_lma + b) - y_lma)**2)

def rat(ab):
    a, b = ab
    s = 0
    for k in range(0, 101):
        s += (a / (1+x[k]*b)- y[k]) ** 2
    return s
def rat_jac(ab):
    a, b = ab
    l = 0
    r = 0
    s = 0
    for k in range(0, 101):
        l += ((a/(1+b*x[k]) - y[k])*2/(1+b*x[k]))
        r += (2*a*x[k]/(1+b*x[k])**2 * (a/(1+b*x[k])-y[k]))
    s = np.array([l, r])
    return s
def rat_hess(ab):
    a, b = ab
    hess = np.ones([2,2])
    lh = 0
    rh = 0
    ld = 0
    rd = 0
    for k in range(0, 101):
        lh += (2 / (1+b*x[k])**2)
        rh += (-2*a*x[k]/(1+b*x[k])**3-2*x[k]*(a/(1+b*x[k])-y[k]) / (1+b*x[k])**2)
        ld += (-2*a*x[k]/(1+b*x[k])**3-2*x[k]*(a/(1+b*x[k])-y[k])/(1+b*x[k])**3)
        rd += (2 * a**2 * x[k]**2 / (1+b*x[k])**4 * 4 * a * x[k]**2 * (a/(1+b*x[k])-y[k]) / (1+b*x[k])**3)
    hess[0,0] = lh
    hess[0,1] = rh
    hess[1,0] = ld
    hess[1,1] = rd
    return hess
def rat_lma(ab):
    a, b = ab
    global x_lma, y_lma
    return ( ((a / ( 1 + b * x_lma )) - y_lma)**2)


ex_2_lin = minimize(lin, [1., 1.], method='CG', options={'xtol': 1e-3, 'disp':True})
ex_2_lin = ex_2_lin.x
ex_2_rat = minimize(rat, [1., -0.5], method='CG', options={'xtol': 1e-3, 'disp':True})
ex_2_rat = ex_2_rat.x 
# ex_2_lin = conjugate_gradient(lin, lin_jac, [1., 1.], max_iter=1000)
# ex_2_rat = conjugate_gradient(rat, rat_jac, [1, -0.1], max_iter=10000)

#Newton algorithm
ex_3_lin = minimize(lin, [1., 1.], method='Newton-CG', jac=lin_jac, hess=lin_hess, options={'xtol': 1e-3, 'disp':True})
ex_3_rat = minimize(rat, [1., -0.5], method='Newton-CG', jac=rat_jac, hess=rat_hess, options={'xtol': 1e-3, 'disp':True})

# Levenberg-Marquardt algorithm (LMA)
ex_4_lin = optimize.least_squares(lin_lma, [1., 1.], method="lm")
ex_4_rat = optimize.least_squares(rat_lma, [1., 1.], method="lm")

#Nedler
result1 = minimize(lin,[0.3, 0.3], method='nelder-mead', options={'xatol': 0.001,'disp': True})
result2 = minimize(rat,[0.3, 0.3], method='nelder-mead', options={'xatol': 0.001,'disp': True})

Optimization terminated successfully.
         Current function value: 1.329315
         Iterations: 2
         Function evaluations: 15
         Gradient evaluations: 5
Optimization terminated successfully.
         Current function value: 1.312175
         Iterations: 10
         Function evaluations: 68
         Gradient evaluations: 22
Optimization terminated successfully.
         Current function value: 1.329955
         Iterations: 32
         Function evaluations: 34
         Gradient evaluations: 34
         Hessian evaluations: 32
         Current function value: 2.866744
         Iterations: 1
         Function evaluations: 24
         Gradient evaluations: 12
         Hessian evaluations: 2
Optimization terminated successfully.
         Current function value: 1.329319
         Iterations: 33
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: 1.312183
         Iterations: 31
         Function evaluations: 62


/var/folders/7f/v45zg7ms5gzd2qjc6tqr5nwh0000gn/T/ipykernel_22059/3080267974.py:78: OptimizeWarning: Unknown solver options: xtol
  ex_2_lin = minimize(lin, [1., 1.], method='CG', options={'xtol': 1e-3, 'disp':True})
/var/folders/7f/v45zg7ms5gzd2qjc6tqr5nwh0000gn/T/ipykernel_22059/3080267974.py:80: OptimizeWarning: Unknown solver options: xtol
  ex_2_rat = minimize(rat, [1., -0.5], method='CG', options={'xtol': 1e-3, 'disp':True})


In [5]:

def table_output_linear ():
    matplotlib.use('TkAgg')
    top = Toplevel(gui)
    fig = plt.figure()
    # Special type of "canvas" to allow for matplotlib graphing
    canvas = matplotlib.backends.backend_tkagg.FigureCanvasTkAgg(fig, master=top)
    plot_widget = canvas.get_tk_widget()
    plt.scatter(x,y)
    plt.plot(xk, ex_1_lin[0]*xk+ex_1_lin[1], 'r', label='gd alg')
    plt.plot(xk, ex_2_lin[0]*xk+ex_2_lin[1], 'g', label='cgd alg')
    plt.plot(xk, ex_3_lin.x[0]*xk+ex_3_lin.x[1], 'y', label='newton alg')
    plt.plot(xk, ex_4_lin.x[0]*xk+ex_4_lin.x[1], 'b', label='lma')
    plt.plot(xk, result1.x[0]*xk+result1.x[1], 'y', label='nedler alg')
    plt.ylabel('y(x) = α‎ * x(k) + β + δ(k)', fontsize=12)
    plt.xlabel('x(k) = k / 100, k = [0; 1]', fontsize=12)
    plt.legend(fontsize=10)
    plot_widget.grid(row=0, column=0)
    top.mainloop()
def table_output_rational ():
    matplotlib.use('TkAgg')
    top = Toplevel(gui)
    fig = plt.figure()
    # Special type of "canvas" to allow for matplotlib graphing
    canvas = matplotlib.backends.backend_tkagg.FigureCanvasTkAgg(fig, master=top)
    plot_widget = canvas.get_tk_widget()
    plt.scatter(x,y)
    plt.plot(xk, ex_1_rat[0] / (1 + ex_1_rat[1] * xk), 'r', label='gd alg')
    plt.plot(xk, ex_2_rat[0] / (1 + ex_2_rat[1] * xk), 'g', label='cgd alg')
    plt.plot(xk, ex_3_rat.x[0] / (1 + ex_3_rat.x[1] * xk), 'y', label='newton alg')
    plt.plot(xk, ex_4_rat.x[0] / (1 + ex_4_rat.x[1] * xk), 'b', label='lma')
    plt.plot(xk, result2.x[0] / (1 + result2.x[1] * xk), 'y', label='nedler alg')
    plt.ylabel('y(x) = α‎ * x(k) + β + δ(k)', fontsize=12)
    plt.xlabel('x(k) = k / 100, k = [0; 1]', fontsize=12)
    plt.legend(fontsize=10)
    plot_widget.grid(row=0, column=0)
    top.mainloop()

#GUI
gui = Tk ()
gui.title('Alogorithms for unconstrained non-linear optimization')
gui.geometry("300x300")

#buttons
label1 = Label ( text = 'Ex.2. y(x) = α‎ * x(k) + β + δ(k), x(k) = k / 100 \nChoose a plot below', fg='black', bg='azure', width=20, height=5)
label1.pack(fill=tk.X)
btn4_1 = Button (text = 'a* x(k) + b', fg='black', bg='lemon chiffon', width=20, height=5, command=table_output_linear)
btn4_1.pack(fill=tk.X)
btn4_2 = Button (text = 'a / (x(k) * b + 1)', fg='black', bg='lemon chiffon', width=20, height=5, command=table_output_rational)
btn4_2.pack(fill=tk.X)

gui.mainloop()

In [6]:
#Old methods from Ex.2
arr_ex_1_2_x = []
arr_ex_3_x = []
arr_ex_1 = []
arr_ex_2 = []
arr_ex_3 = []
arr_1_ex_4 = []
arr_2_ex_4 = []
y_ex_4 = []
x_ex_4 = []
b = []
a_1=0
b_1=0
a_2=0
b_2=0
a_3 = 0.05
b_3 = 0.05
b_3_temp = 10000
a_4 = 0.05
b_4 = 0.05
b_4_temp = 10000
#gauss alghorithm for 2 part
while 0 == 0:
    ex_4_gauss_arr_1 = []
    ex_4_gauss_arr_2 = []
    for a in range(0, 1000):
        a = a / 1000
        s = 0
        for k in range(0, 101):
            s += (a * x_ex_4[k] + b_3 - y_ex_4[k])**2
        ex_4_gauss_arr_1.append(s)
    a_3_temp = ex_4_gauss_arr_1.index(min(ex_4_gauss_arr_1)) / 1000
    if abs(a_3 - a_3_temp) < 0.001 and abs(b_3 - b_3_temp) < 0.001:
        break
    a_3 = a_3_temp
    for b in range(0, 1000):
        b = b / 1000
        s = 0
        for k in range(0, 101):
            s += (a_3 * x_ex_4[k] + b - y_ex_4[k])**2
        ex_4_gauss_arr_2.append(s)
    b_3_temp = ex_4_gauss_arr_2.index(min(ex_4_gauss_arr_2)) / 1000
    if abs(b_3 - b_3_temp) < 0.001 and abs(a_3 - a_3_temp) < 0.001:
        break
    b_3 = b_3_temp
while 0 == 0:
    ex_4_gauss_arr_1 = []
    ex_4_gauss_arr_2 = []
    for a in range(0, 1000):
        a = a / 1000
        s = 0
        for k in range(0, 101):
            s += (a / (1 + b_4 * x_ex_4[k]) - y_ex_4[k])**2
        ex_4_gauss_arr_1.append(s)
    a_4_temp = ex_4_gauss_arr_1.index(min(ex_4_gauss_arr_1)) / 1000
    if abs(a_4 - a_4_temp) < 0.001 and abs(b_4 - b_4_temp) < 0.001:
        break
    a_4 = a_4_temp
    for b in range(0, 1000):
        b = - b / 1000
        s = 0
        for k in range(0, 101):
            s += (a_4 / (1 + b * x_ex_4[k]) - y_ex_4[k])**2
        ex_4_gauss_arr_2.append(s)
    b_4_temp = ex_4_gauss_arr_2.index(min(ex_4_gauss_arr_2)) / -1000
    if abs(b_4 - b_4_temp) < 0.001 and abs(a_4 - a_4_temp) < 0.001:
        break
    b_4 = b_4_temp

#brutforce algorithm for 2 part
for a in range(0, 1000):
    a = a / 1000
    for b in range(0, 1000):
        b1 = b / 1000
        b2 = - b / 1000
        s1 = 0
        s2 = 0
        for k in range(0, 101):
            s1 += (a * x_ex_4[k] + b1 - y_ex_4[k]) ** 2
            s2 += (a / (1 + b2 * x_ex_4[k]) - y_ex_4[k]) ** 2
        arr_1_ex_4.append([s1, a, b1])
        arr_2_ex_4.append([s2, a, b2])
minimum_1=min(x1[0] for x1 in arr_1_ex_4)
minimum_2=min(x2[0] for x2 in arr_2_ex_4)
for sublist in arr_1_ex_4:
    if sublist[0] == minimum_1:
        a_1 = sublist[1]
        b_1 = sublist[2]
        break
for sublist in arr_2_ex_4:
    if sublist[0] == minimum_2:
        a_2 = sublist[1]
        b_2 = sublist[2]
        break



IndexError: list index out of range